The image files and the trimaps need to be loaded and stored in the respective folders before the processing starts.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [2]:
!mkdir original_img
!mv GT* original_img/.

In [6]:
!mkdir trimap_img
!mv GT* trimap_img/.

In [8]:
!mkdir results
!mv GT* results/.

In [ ]:
!mkdir results_exact
!mv GT* results_exact/.

In [4]:
!mkdir ground_truth
!mv GT* ground_truth/.

# Image Matting


In [ ]:
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

def gaussian_weight_matrix(N, sigma):
    m, n = [s//2 for s in N]
    x, y = np.ogrid[-m:m+1, -n:n+1]
    h = np.exp(-(x*x + y*y)/(2.*sigma*sigma))
    #To restrict the precision errors, very small values are approximated with 0
    h[h < 2e-16] = 0
    if (np.sum(h)!=0):
      h = h/np.sum(h)
    return h

def neighbors(src, x, y, N):
  size = src.shape
  h = size[0]
  w = size[1]
  n = N//2
  low_lim_x = max(0,x-n)
  low_lim_y = max(0,y-n)
  up_lim_x = min(w,x+1+n)
  up_lim_y = min(h,y+1+n)
  
  if len(size)==2:
    #For alpha matrix
    neigh = np.zeros((N,N))
    neigh[n-(y-low_lim_y):n+(up_lim_y-y),n-(x-low_lim_x):n+(up_lim_x-x)] = src[low_lim_y:up_lim_y,low_lim_x:up_lim_x]
  else:
    #For foreground and background matrix
    neigh = np.zeros((N,N,3))
    neigh[n-(y-low_lim_y):n+(up_lim_y-y),n-(x-low_lim_x):n+(up_lim_x-x),:] = src[low_lim_y:up_lim_y,low_lim_x:up_lim_x]
  return neigh

def characterize(x, w):
  W = np.sum(w)
  #Mean vector
  mu = np.einsum('ij,i->j', x, w)/W
  #Normalizing the vectors
  norm = x-mu[:,np.newaxis].T
  weighted_norm = np.einsum('ij,i->ij', norm, np.sqrt(w))
  #Computing weighted covariance matrix
  #The eignevectors are same for Cov matrix computed as X*X.T or X.T*X
  #Extra error component added to prevent Singular matrices
  cov = (weighted_norm.T @ weighted_norm)/W + 1e-5*np.eye(3)
  #Finding the eigenvalues of the covariance matrix, for clustering
  val, vec = np.linalg.eig(np.nan_to_num(cov))
  cluster = {}
  cluster["x"] = x
  cluster["w"] = w
  cluster["Mean"] = mu
  cluster["Cov"] = cov
  cluster["Eigenvalues"] = val
  cluster["Eigenvectors"] = vec
  cluster["Sig_val"] = np.max(np.abs(val))
  cluster["Sig_vec"] = vec[np.argmax(np.abs(val))]
  return cluster


def cluster(x, w, dis=0.05):
  mu = []
  sigma = []
  clusters = []

  cluster = characterize(x, w)
  #Determining the mean and covariance for the cluster
  clusters.append(cluster)

  #dis is the minimum variance between two clusters
  #iterated till any cluster has two elements which are more distant than dis
  while max(clusters, key=lambda x: x["Sig_val"])["Sig_val"] > dis:
    idx_max = max(enumerate(clusters), key=lambda x: x[1]["Sig_val"])[0]
    C_i = clusters[idx_max]
    idx = C_i["x"] @ C_i["Sig_vec"] <= np.dot(C_i["Mean"], C_i["Sig_vec"])
    #Separating the elements into two clusters 
    #Based on their component in the direction of Sig_vec being smaller and larger than that of the mean vector
    C_a = characterize(C_i["x"][idx], C_i["w"][idx])
    C_b = characterize(C_i["x"][np.logical_not(idx)], C_i["w"][np.logical_not(idx)])
    #Defining two new clusters, instead of the old one
    clusters.pop(idx_max)
    clusters.append(C_a)
    clusters.append(C_b)
  
  for n,cluster in enumerate(clusters):
    mu.append(cluster["Mean"])
    sigma.append(cluster["Cov"])
  
  return np.array(mu), np.array(sigma)

def solve(F_bar, F_Cov, B_bar, B_Cov, C, sigma_C, alpha_init, maxIter, minLike):
  alpha = alpha_init
  Max_alpha = -1

  last_like = -np.inf
  chg_like = minLike + 1
  max_like = -np.inf

  A = np.zeros((2*3,2*3))
  b = np.zeros((1,6))
  I = np.eye(3)
  inv_sigma_C_2 = 1/(sigma_C*sigma_C)
  
  for i in range(np.size(F_bar,0)):
    F_bar_i = F_bar[i]
    F_Sigma_inv_i = np.linalg.inv(F_Cov[i])

    for j in range(np.size(B_bar,0)):
      B_bar_j = B_bar[j]
      B_Sigma_inv_j = np.linalg.inv(B_Cov[j])

      iter = 1
      while ((iter < maxIter)&(chg_like > minLike)):
          
          A[0:3,0:3] = F_Sigma_inv_i + I*alpha**2 * inv_sigma_C_2
          A[0:3,3:6] = I*alpha*(1-alpha) * inv_sigma_C_2
          A[3:6,3:6] = B_Sigma_inv_j + I*(1-alpha)**2 * inv_sigma_C_2
          A[3:6,0:3] = A[0:3,3:6]
          b[0,0:3] = np.matmul(F_Sigma_inv_i, F_bar_i) + C * alpha * inv_sigma_C_2
          b[0,3:6] = np.matmul(B_Sigma_inv_j, B_bar_j) + C * (1-alpha) * inv_sigma_C_2

          X = np.linalg.solve(A, b.T)
          F = X[0:3]
          B = X[3:6]
          F = np.maximum(0, np.minimum(1, X[0:3]))
          B = np.maximum(0, np.minimum(1, X[3:6]))
          
                # solve for alpha

          alpha = np.matmul((np.atleast_2d(C).T-B).T, (F-B)) / np.sum((F-B)**2)
          alpha = np.maximum(0, np.minimum(1, alpha))[0,0]

                # # calculate likelihood
          L_C = - np.sum((np.atleast_2d(C).T -alpha*F-(1-alpha)*B)**2) * inv_sigma_C_2
          L_F = (-1* np.matmul(np.matmul((F- np.atleast_2d(F_bar_i).T).T , F_Sigma_inv_i), (F-np.atleast_2d(F_bar_i).T))/2)[0,0]
          L_B = (-1* np.matmul(np.matmul((B- np.atleast_2d(B_bar_j).T).T , B_Sigma_inv_j), (B-np.atleast_2d(B_bar_j).T))/2)[0,0]
          like = (L_C + L_F + L_B)
                #like = 0

          if (like > max_like):
            max_like = like
            Max_alpha = alpha
            Max_F = F.ravel()
            Max_B = B.ravel()

          chg_like = like-last_like
          iter = iter+1
  
  return Max_F, Max_B, Max_alpha

def matting(img_path, trimap_path):
  #Reading the image and trimap file 
  #(and mapping the pixel values from 0-255 to 0-1, to maintain consistency in order of values in computation)
  img = np.array(cv2.imread(img_path))/255
  trimap = np.array(cv2.imread(trimap_path, cv2.IMREAD_GRAYSCALE))

  h,w,c = img.shape
  #segregating the foreground and background (already known pixels)
  fg_mask = (trimap == 255)
  bg_mask = (trimap == 0)
  fg_pix = np.zeros((h,w,3))
  bg_pix = np.zeros((h,w,3))
  for i in range(3):
    fg_pix[:,:,i] = img[:,:,i]*fg_mask
    bg_pix[:,:,i] = img[:,:,i]*bg_mask

  #alpha is initialized with 1-foreground, 0-background, nan-unknown pixels
  alpha = np.array(fg_mask, dtype=float)
  uk_mask = np.logical_not(np.logical_or(fg_mask,bg_mask))
  alpha[uk_mask] = np.nan

  #unknown pixels separated out  
  n_unknown = np.sum(uk_mask)
  unknown = uk_mask
  
  #determining the size and weights for the sliding window
  #Sigma is taken to be 8, and mean 0 for the gaussian distribution
  Min_neighbor = 10
  #again mapping the weights to the range 0-1 
  
  #Strip size to work on, in one iteration
  border = np.ones((3,3))
  n_decided = 1
  N = 25
  while n_decided < n_unknown:
    print("Unknown: "+str(n_unknown-n_decided))
    #Processing the pixels from outward to inward
    unknown = cv2.erode(unknown.astype(np.uint8), border, iterations=1)
    uk_pix = np.logical_and(np.logical_not(unknown), uk_mask)
    #Taking unknowns in the selected strip
    Y, X = np.nonzero(uk_pix) 
    
    #For each unknown pixel
    skip_count = 0
    count = 0
    for i in range(np.size(X,0)):
        if (count == skip_count):
          if (count == (n_unknown-n_decided)):
          #In case the sliding window is insufficient for the process to proceed, increase it marginally
            N = N+2
            print("N incremented to "+str(N))
            count = 0
            skip_count = 0
        else:
          count = 0
          skip_count = 0

        count = count+1
        
        x, y = X[i], Y[i]
        pix = img[y, x]
        
        #Getting the neighbouring pixels in the sliding window
        a = neighbors(alpha, x, y, N)
        fg_pixels = neighbors(fg_pix, x, y, N)
        bg_pixels = neighbors(bg_pix, x, y, N)
        
        #Assigning the weights to these pixel values, and flattening it for the clustering algorithm
        #w_i = (alpha_i^2) * g_i
        gaussian_weights = gaussian_weight_matrix((N,N),8)
        gaussian_weights/np.max(gaussian_weights)

        fg_weights = (a**2 * gaussian_weights).reshape(-1)
        fg_pixels = np.reshape(fg_pixels, (N*N, 3))
        #only non-zero values are computationally useful
        valid_pix = np.nan_to_num(fg_weights) > 0
        fg_pixels = fg_pixels[valid_pix, :]
        fg_weights = fg_weights[valid_pix]      
        
        bg_weights = ((1-a)**2 * gaussian_weights).reshape(-1)
        bg_pixels = np.reshape(bg_pixels, (N*N, 3))
        valid_pix = np.nan_to_num(bg_weights) > 0
        bg_pixels = bg_pixels[valid_pix, :]
        bg_weights = bg_weights[valid_pix]

        #Atleast Min_neighbor pixel values should be there for computation
        if len(fg_weights) < Min_neighbor or len(bg_weights) < Min_neighbor:
            skip_count = skip_count+1
            continue

        #Clustering
        fg_bar, fg_Cov = cluster(fg_pixels, fg_weights)
        bg_bar, bg_Cov = cluster(bg_pixels, bg_weights)

        #Solving by the iterative method
        alpha_init = np.nanmean(a.reshape(-1))
        #Maximum number of iterations is kept as 50 
        #or the iteration woudl stop if change in likelihood is less than 1e-6 between two consecutive steps
        f, b, alphaT = solve(fg_bar, fg_Cov, bg_bar, bg_Cov, pix, 0.01, alpha_init, 50, 1e-6)

        fg_pix[y, x] = f.ravel()
        bg_pix[y, x] = b.ravel()
        alpha[y, x] = alphaT
        uk_mask[y, x] = 0 
        n_decided += 1
        # N = 25
        if (n_decided % 50000 == 0):
          # print(str(n_decided)+" Pixels decided...........")
          cv2_imshow(alpha*255)
          cv2.imwrite("/content/results/GT" + "{:02d}".format(file_no+1) + ".png",alpha)
  return alpha

for file_no in range(17,18):
  img_path = "/content/original_img/GT" + "{:02d}".format(file_no+1) + ".png"
  trimap_path = "/content/trimap_img/GT" + "{:02d}".format(file_no+1) + ".png"
  alpha = matting(img_path, trimap_path)*255
  
  gt_path = "/content/ground_truth/GT" + "{:02d}".format(file_no+1) + ".png"
  gt = np.array(cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE))
  cv2.imwrite("/content/results/GT" + "{:02d}".format(file_no+1) + ".png",alpha)
  
  np.savetxt("/content/results_exact/GT" + "{:02d}".format(file_no+1) + ".txt",alpha)
  abs_diff = np.sum(np.abs(alpha-gt))
  abs_diff_per_pix = abs_diff/(np.size(alpha,0)*np.size(alpha,1))
  
  cv2_imshow(gt)
  print("Ground Truth")
  cv2_imshow(alpha)
  print("Predicted Alpha Matte")
  print("Absolute Diff: "+str(abs_diff))
  print("Absolute Diff Per Pixel: "+str(abs_diff_per_pix))

Visualising the result and computing the quality metric on the results of the processing.

In [ ]:
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

for file_no in range(0,27):
  img_path = "/content/original_img/GT" + "{:02d}".format(file_no+1) + ".png"
  img = np.array(cv2.imread(img_path))
  cv2_imshow(img)
  print("Original Image")
  
  gt_path = "/content/ground_truth/GT" + "{:02d}".format(file_no+1) + ".png"
  gt = np.array(cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE))
  cv2_imshow(gt)
  print("Ground Truth")

  # alpha_path = "/content/results/GT" + "{:02d}".format(file_no+1) + ".png"
  # alpha = np.array(cv2.imread(alpha_path, cv2.IMREAD_GRAYSCALE))
  # cv2_imshow(alpha)
  # print("Predicted Alpha Matte")

  alpha_path_exact = "/content/results_exact/GT" + "{:02d}".format(file_no+1) + ".txt"
  alpha_exact = np.loadtxt(alpha_path_exact,dtype=float)
  cv2_imshow(alpha_exact)
  print("Predicted Alpha Matte Exact")

  diff = np.array(np.abs(gt-alpha_exact),dtype=int)
  if (np.sum(diff)<2e-16):
    print("Mean Absolute Difference (Exact): 0.00")
  else:
    print("Mean Absolute Difference (Exact): " + str(np.sum(diff)/(np.size(alpha_exact,0)*np.size(alpha_exact,1))))
  input()

The following code can be used to extract the foreground from the image with the alpha matte created.

In [ ]:
file_no = 17
img_path = "/content/original_img/GT" + "{:02d}".format(file_no+1) + ".png"
img = np.array(cv2.imread(img_path))
cv2_imshow(img)
print("Original Image")
  
gt_path = "/content/ground_truth/GT" + "{:02d}".format(file_no+1) + ".png"
gt = np.array(cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE),dtype=float)
cv2_imshow(gt)
print("Ground Truth")

extract_with_gt = (img*gt[:,:,np.newaxis])/255
cv2_imshow(extract_with_gt)
print("Extracted with the Ground Truth")

alpha_path_exact = "/content/results_exact/GT" + "{:02d}".format(file_no+1) + ".txt"
alpha_exact = np.loadtxt(alpha_path_exact,dtype=float)
cv2_imshow(alpha_exact)
print("Predicted Alpha Matte Exact")

extract_with_pred = (img*alpha_exact[:,:,np.newaxis])/255
cv2_imshow(extract_with_pred)
print("Extracted with the Predicted Alpha Matte")

cv2.imwrite("/content/GT_with_gt" + "{:02d}".format(file_no+1) + ".png",extract_with_gt)
cv2.imwrite("/content/GT_with_pred" + "{:02d}".format(file_no+1) + ".png",extract_with_pred)